# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
# List all the database names
database_names = mongo.list_database_names()

# Check if the 'uk_food' database is in the list of database names
if 'uk_food' in database_names:
    print("The 'uk_food' database was created.")
else:
    print("The 'uk_food' database was not found.")

print (database_names)

The 'uk_food' database was created.
['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
# List all the collections in the 'uk_food' database
collection_names = db.list_collection_names()

# Print the collection names
for collection_name in collection_names:
    print(collection_name)

establishments


In [ ]:
# review the collections in our new database


In [ ]:
# review a document in the establishments collection


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [7]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}



In [8]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant_data)

In [9]:
# Check that the new restaurant was inserted
print(establishments.find_one({"name": "Penang Flavours"}))
    

{'_id': ObjectId('650a5cea365c679890e8cdbb'), 'name': 'Penang Flavours', 'address': {'street': '123 Main Street', 'city': 'Greenwich', 'state': 'London', 'postcode': 'SE10 8LL'}, 'cuisine': 'Halal', 'rating': None, 'BusinessTypeID': '', 'geocode': {'longitude': None, 'latitude': None}, 'RatingValue': None}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}
# Projection to return only BusinessTypeID and BusinessType fields
Fields = {
 
    "BusinessTypeID": 1,
    "BusinessType": 1
}

business_type_result = establishments.find_one(business_type_query, Fields)
print(business_type_result)

{'_id': ObjectId('650a652b185253b6f2623265'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
new_restaurant_query = {"name": "Penang Flavours"}
new_business_type_id = business_type_result.get("BusinessTypeID")

update_query = {"$set": {"BusinessTypeID": new_business_type_id}}

update_result = establishments.update_one(new_restaurant_query, update_query)

In [13]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("The new restaurant 'Penang Flavours' was updated with the correct BusinessTypeID.")
else:
    print("There was an issue updating the new restaurant.")

There was an issue updating the new restaurant.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
dover_count = establishments.count_documents(dover_query)
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")

Number of documents with LocalAuthorityName as 'Dover': 994


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many(dover_query)
print(f"Deleted {delete_result.deleted_count} documents with LocalAuthorityName as 'Dover'")

Deleted 994 documents with LocalAuthorityName as 'Dover'


In [16]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents(dover_query)
print(f"Number of remaining documents with LocalAuthorityName as 'Dover': {remaining_dover_count}")

Number of remaining documents with LocalAuthorityName as 'Dover': 0


In [17]:
# Check that other documents remain with 'find_one'
sample_document = establishments.find_one()
if sample_document:
    print("Sample document from the collection:")
    pprint(sample_document)
else:
    print("No documents found in the collection.")

Sample document from the collection:
{'BusinessTypeID': '',
 'RatingValue': None,
 '_id': ObjectId('650a5cea365c679890e8cdbb'),
 'address': {'city': 'Greenwich',
             'postcode': 'SE10 8LL',
             'state': 'London',
             'street': '123 Main Street'},
 'cuisine': 'Halal',
 'geocode': {'latitude': None, 'longitude': None},
 'name': 'Penang Flavours',
 'rating': None}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal

establishments.update_many({}, [
    {
        "$set": {
            "geocode.longitude": {
                "$toDouble": "$geocode.longitude"
            },
            "geocode.latitude": {
                "$toDouble": "$geocode.latitude"
            }
        }
    }
])

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {
        "$set": {
            "RatingValue": {
                "$toInt": "$RatingValue"
            }
        }
    }
])

In [21]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()
if sample_document:
    print("Sample document from the collection:")
    pprint(sample_document)
else:
    print("No documents found in the collection.")

Sample document from the collection:
{'BusinessTypeID': '',
 'RatingValue': None,
 '_id': ObjectId('650a5cea365c679890e8cdbb'),
 'address': {'city': 'Greenwich',
             'postcode': 'SE10 8LL',
             'state': 'London',
             'street': '123 Main Street'},
 'cuisine': 'Halal',
 'geocode': {'latitude': None, 'longitude': None},
 'name': 'Penang Flavours',
 'rating': None}
